In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd
import csv
import re

In [65]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt

In [67]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
device = torch.device('cuda')

In [69]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [70]:
get_ipython().system('pip install gensim')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import pickle
import gensim

In [72]:
model = KeyedVectors.load_word2vec_format('/content/gdrive/MyDrive/Projects/Stance Detection with Bidirectional Conditional Encoding/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary = True)

In [73]:
train_df = pd.read_csv('/content/gdrive/MyDrive/Projects/Stance Detection with Bidirectional Conditional Encoding/dataset/train.csv', engine = 'python', encoding = 'latin')
test_df = pd.read_csv('/content/gdrive/MyDrive/Projects/Stance Detection with Bidirectional Conditional Encoding/dataset/test.csv', engine = 'python', encoding = 'latin')

In [74]:
print(train_df.head())
print(test_df.head())
print(train_df.shape)
print(test_df.shape)

                                               Tweet           Target  \
0  @tedcruz And, #HandOverTheServer she wiped cle...  Hillary Clinton   
1  Hillary is our best choice if we truly want to...  Hillary Clinton   
2  @TheView I think our country is ready for a fe...  Hillary Clinton   
3  I just gave an unhealthy amount of my hard-ear...  Hillary Clinton   
4  @PortiaABoulger Thank you for adding me to you...  Hillary Clinton   

    Stance                                    Opinion Towards Sentiment  
0  AGAINST  1.  The tweet explicitly expresses opinion abo...       neg  
1    FAVOR  1.  The tweet explicitly expresses opinion abo...       pos  
2  AGAINST  1.  The tweet explicitly expresses opinion abo...       neg  
3  AGAINST  1.  The tweet explicitly expresses opinion abo...       neg  
4     NONE  3.  The tweet is not explicitly expressing opi...       pos  
                                               Tweet   Target   Stance  \
0  He who exalts himself shall      be humb

In [75]:
train_df = train_df.drop(columns=['Opinion Towards', 'Sentiment'])
train_df = train_df.dropna()
train_df

,Tweet,Target,Stance
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE
...,...,...,...
2909,"There's a law protecting unborn eagles, but no...",Legalization of Abortion,AGAINST
2910,I am 1 in 3... I have had an abortion #Abortio...,Legalization of Abortion,AGAINST
2911,How dare you say my sexual preference is a cho...,Legalization of Abortion,AGAINST
2912,"Equal rights for those 'born that way', no rig...",Legalization of Abortion,AGAINST


In [76]:
test_df = test_df.drop(columns = ['Opinion Towards', 'Sentiment'])
test_df = test_df.dropna()
test_df.head()

,Tweet,Target,Stance
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST
3,#God is utterly powerless without Human interv...,Atheism,AGAINST
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST


In [77]:
test_df = test_df[test_df['Target'] == 'Donald Trump']
val_df = train_df[train_df['Target'] == 'Hillary Clinton']
train_df = train_df[train_df['Target'] != 'Hillary Clinton']

In [78]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(2225, 3)
(707, 3)
(689, 3)


In [79]:
print(train_df.head())
print(test_df.head())

                                                Tweet  \
50  Just laid down the law on abortion in my bioet...   
51  @tooprettyclub Are you OK with #GOP males tell...   
52  If you don't want your kid, put it up for adop...   
53  @RedAlert -there should be a "stigma" to butch...   
54  But isn't that the problem then. Not enough fa...   

                      Target   Stance  
50  Legalization of Abortion  AGAINST  
51  Legalization of Abortion    FAVOR  
52  Legalization of Abortion  AGAINST  
53  Legalization of Abortion  AGAINST  
54  Legalization of Abortion     NONE  
                                                  Tweet        Target   Stance
1249  @2014voteblue @ChrisJZullo blindly supporting ...  Donald Trump     NONE
1250  @ThePimpernelX @Cameron_Gray @CalebHowe  Total...  Donald Trump     NONE
1251  @JeffYoung @ThePatriot143 I fully support full...  Donald Trump     NONE
1252  @ABC Stupid is as stupid does! Showedhis true ...  Donald Trump  AGAINST
1253  @HouseGOP we now

In [80]:
print(train_df['Stance'].value_counts())
print(test_df['Stance'].value_counts())
print(val_df['Stance'].value_counts())

AGAINST    1002
FAVOR       635
NONE        588
Name: Stance, dtype: int64
AGAINST    299
NONE       260
FAVOR      148
Name: Stance, dtype: int64
AGAINST    393
NONE       178
FAVOR      118
Name: Stance, dtype: int64


In [81]:
remove_against = 1002 - 588

In [82]:
against_df = train_df[train_df['Stance'] == 'AGAINST']
none_df = train_df[train_df['Stance'] == 'NONE']
favor_df = train_df[train_df['Stance'] == 'FAVOR']

In [83]:
against_drop_indices = np.random.choice(against_df.index, remove_against, replace = False)
against_undersampled = against_df.drop(against_drop_indices)

In [84]:
against_undersampled

,Tweet,Target,Stance
50,Just laid down the law on abortion in my bioet...,Legalization of Abortion,AGAINST
52,"If you don't want your kid, put it up for adop...",Legalization of Abortion,AGAINST
53,"@RedAlert -there should be a ""stigma"" to butch...",Legalization of Abortion,AGAINST
63,Planned Parenthood lines its pockets while sac...,Legalization of Abortion,AGAINST
65,"RT @TheKeyisPrayer: ""No law can give me the ri...",Legalization of Abortion,AGAINST
...,...,...,...
2901,"#Love does not delight in #evil, but #rejoices...",Legalization of Abortion,AGAINST
2908,When you bring up the 50 million black babies ...,Legalization of Abortion,AGAINST
2909,"There's a law protecting unborn eagles, but no...",Legalization of Abortion,AGAINST
2910,I am 1 in 3... I have had an abortion #Abortio...,Legalization of Abortion,AGAINST


In [85]:
balanced_train_df = pd.concat([favor_df, against_undersampled, none_df])

In [86]:
balanced_train_df['Stance'].value_counts()

FAVOR      635
AGAINST    588
NONE       588
Name: Stance, dtype: int64

In [87]:
balanced_train_df['Stance'] = balanced_train_df['Stance'].map({
    'AGAINST' : [1, 0, 0],
    'FAVOR' : [0, 1, 0],
    'NONE' : [0, 0, 1]
})

In [88]:
test_df['Stance'] = test_df['Stance'].map({
    'AGAINST' : [1, 0, 0],
    'FAVOR' : [0, 1, 0],
    'NONE' : [0, 0, 1]
})

In [89]:
val_df['Stance'] = val_df['Stance'].map({
    'AGAINST' : [1, 0, 0],
    'FAVOR' : [0, 1, 0],
    'NONE' : [0, 0, 1]
})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [90]:
val_df

,Tweet,Target,Stance
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,"[1, 0, 0]"
1,Hillary is our best choice if we truly want to...,Hillary Clinton,"[0, 1, 0]"
2,@TheView I think our country is ready for a fe...,Hillary Clinton,"[1, 0, 0]"
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,"[1, 0, 0]"
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,"[0, 0, 1]"
...,...,...,...
2306,Today's number is 8! That's how many years old...,Hillary Clinton,"[1, 0, 0]"
2307,@JessieJaneDuff Results matter. U may feel les...,Hillary Clinton,"[1, 0, 0]"
2308,Grateful for the chance to tell the world abou...,Hillary Clinton,"[1, 0, 0]"
2309,"If you cant dazzle them with brilliance, baffl...",Hillary Clinton,"[1, 0, 0]"


In [91]:
balanced_train_df.shape

(1811, 3)

In [92]:
train_set = list(balanced_train_df.to_records(index = False))
test_set = list(test_df.to_records(index = False))
val_set = list(val_df.to_records(index = False))

In [93]:
def remove_links_mentions(word):
  link_re_pattern = 'https?:\/\/t.co/[\w]+'
  mention_re_pattern = '@\w+'
  word = re.sub(link_re_pattern, '', word)
  word = re.sub(mention_re_pattern, '', word)
  word = re.sub(r'[?|!|\'|"|#]', r'', word)
  word = re.sub(r',[.|,|)|(|\|/]', r' ', word)
  word = word.replace('_', ' ')
  return word.lower()

In [94]:
cleaned_train_tweet = []
cleaned_train_target = []
for tweet, target, stance in train_set:
  cleaned_train_tweet.append(remove_links_mentions(tweet))

for tweet, target, stance in train_set:
  if target in cleaned_train_target:
    continue
  else:
    cleaned_train_target.append(remove_links_mentions(target))        

In [95]:
cleaned_test_tweet = []
cleaned_test_target = []
for tweet, target, stance in test_set:
  cleaned_test_tweet.append(remove_links_mentions(tweet))

for tweet, target, stance in test_set:
  if (target) in cleaned_test_target:
    continue
  else:
    cleaned_test_target.append(remove_links_mentions(target)) 

In [96]:
cleaned_val_tweet = []
cleaned_val_target = []
for tweet, target, stance in val_set:
  cleaned_val_tweet.append(remove_links_mentions(tweet))

for tweet, target,stance in val_set:
  if (target) in cleaned_val_target:
    continue
  else:
    cleaned_val_target.append(remove_links_mentions(target))

In [97]:
cleaned_train_target = list(cleaned_train_target)
print(len(cleaned_train_target))
print(len(cleaned_train_tweet))
print(cleaned_train_tweet)

1811
1811
[' are you ok with gop males telling you what you can and cant do with your own body', 'lmao my school thinks giving women the right to an abortion is against feminism. they dont know feminism wtf', ' putting abortion in with choices for women just won my heart over. yaaaaaas shahs', 'we live in a world where people care more about religious rights than consent & reproductive rights....', 'my teacher today basically said that embroyos are nothing but skin + blood & that abortion was fine. smartlady', ' keeping pregnant people safe is a great reason for abortion, though.', 'yep. rt: defunding pp wont mean fewer abortions. it will mean more dead women. prochoice 1in3', ' you would force a woman to risk her health, sanity, and life, to continue a doomed pregnancy. against her will. no.', 'whether someone wants to have children or not should be completely up to the person carrying that pregnancy...', 'i watched aftertiller and it gave me a whole new perspective on a womans right 

In [98]:
cleaned_test_target = list(cleaned_test_target)
print(len(cleaned_test_target))
print(len(cleaned_test_tweet))
print(cleaned_test_tweet)

707
707
['  blindly supporting political parties is willfull ignorance, repub or dem. semst', '    totally agree ...time to get out of the twitter peanut gallery and take arms semst', '  i fully support full repeal. i will work for candidates who i believe will make it happen semst', ' stupid is as stupid does showedhis true colors; seems that he ignores that us was invaded, & plundered,not discovered semst', ' we now have one political party. the socialist party. no more bs. we dont believe you anymore. semst', 'donald trump isnt afraid to roast everyone. semst', 'donald trump for president i can dig it.  semst', 'nothing about obamacare changed today what changed is the total exposure of anti-constitutional terrorism at the supremecourt semst', 'i have one friend, he said he has more accountants than he has nurses. semst', 'after today scotus has passed more legislation than congress. tcot semst', '       100% support trump.. semst', '  haha aca is far from affordable you bunch of gr

In [99]:
cleaned_val_target = list(cleaned_val_target)
print(len(cleaned_val_target))
print(len(cleaned_val_tweet))
print(cleaned_val_tweet)

689
689
[' and, handovertheserver she wiped clean + 30k deleted emails, explains dereliction of duty/lies re benghazi,etc tcot', 'hillary is our best choice if we truly want to continue being a progressive nation. ohio', ' i think our country is ready for a female pres, it cant ever be hillary', 'i just gave an unhealthy amount of my hard-earned money away to the big govt & untrustworthy irs. whyimnotvotingforhillary', ' thank you for adding me to your list', 'hillary can not win. heres hoping the dems offer a real candidate like warren. warren2016', 'respect for the law and respect by the law yes, needed desperately.  baltimoreriots', 'i dont want to be appointed to an ambassador post.', 'stophillary2016  if there was a woman with integrity and honesty i would vote for such as woman president, no', ' end lawless clintonfoundation. jail butcher of benghazi. arrest rapist billclinton. hillaryclinton', 'use your brain, keep hillary out of the white house.clinton2016', ' hillary pandering

In [100]:
print(balanced_train_df['Stance'])

51      [0, 1, 0]
57      [0, 1, 0]
58      [0, 1, 0]
66      [0, 1, 0]
69      [0, 1, 0]
          ...    
2822    [0, 0, 1]
2825    [0, 0, 1]
2829    [0, 0, 1]
2830    [0, 0, 1]
2833    [0, 0, 1]
Name: Stance, Length: 1811, dtype: object


In [101]:
import nltk
import time
from nltk.corpus import stopwords
from nltk import word_tokenize
import joblib
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [102]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_tweet_token = []
for data in cleaned_train_tweet:
  new_text = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_text.append(word)  
  cleaned_tweet_token.insert(i, new_text)
  i = i + 1   

In [103]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_target_token = []
for data in cleaned_train_target:
  new_text = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_text.append(word)  
  cleaned_target_token.insert(i, new_text)
  i = i + 1 

In [104]:
print(cleaned_target_token)

[['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['legalization', 'abortion'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'], ['atheism'],

In [105]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_tweet_token_test = []
for data in cleaned_test_tweet:
  new_text = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_text.append(word)  
  cleaned_tweet_token_test.insert(i, new_text)
  i = i + 1 

In [106]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_tweet_token_val = []
for data in cleaned_val_tweet:
  new_val = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_val.append(word)  
  cleaned_tweet_token_val.insert(i, new_text)
  i = i + 1 

In [107]:
print(cleaned_tweet_token_val)

[[',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'free', 'thinkers,1a', 'semst'], [',', 'p.c', '.', 'control', ',', 'stop', 'offended', ',', 'country', 'f

In [108]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_target_token_test = []
for data in cleaned_test_target:
  new_text = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_text.append(word)  
  cleaned_target_token_test.insert(i, new_text)
  i = i + 1  

In [109]:
stop_words = set(stopwords.words('english'))
i = 0
cleaned_target_token_val = []
for data in cleaned_val_target:
  new_val = []
  data = word_tokenize(data)
  for word in data:
    if word not in stop_words:
      new_val.append(word)  
  cleaned_target_token_val.insert(i, new_text)
  i = i + 1  

In [110]:
print(cleaned_target_token_test)

[['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', 'trump'], ['donald', '

In [111]:
print(model.vector_size)
train_tweet = []
count = 0
for words in cleaned_tweet_token:
  vector_list = [model[word] for word in words if word in model.vocab]
  train_tweet.append(vector_list)

from itertools import zip_longest
train_tweet = np.array(list(zip_longest(*train_tweet, fillvalue = np.zeros(300)))).T
train_tweet = train_tweet.transpose([1, 2, 0])
print(train_tweet.shape)

300
(1811, 19, 300)


In [112]:
train_target = []
count = 0
for words in cleaned_target_token:
  vector_list = [model[word] for word in words if word in model.vocab]
  train_target.append(vector_list)

from itertools import zip_longest
train_target = np.array(list(zip_longest(*train_target, fillvalue = np.zeros(300)))).T
train_target = train_target.transpose([1, 2, 0])
print(train_target.shape)

(1811, 4, 300)


In [113]:
test_tweet = []
count = 0
for words in cleaned_tweet_token_test:
  vector_list = [model[word] for word in words if word in model.vocab]
  test_tweet.append(vector_list)

from itertools import zip_longest
test_tweet = np.array(list(zip_longest(*test_tweet, fillvalue = np.zeros(300)))).T
test_tweet = test_tweet.transpose([1, 2, 0])
print(test_tweet.shape)

(707, 19, 300)


In [114]:
val_tweet = []
count = 0
for words in cleaned_tweet_token_val:
  vector_list = [model[word] for word in words if word in model.vocab]
  val_tweet.append(vector_list)

from itertools import zip_longest
val_tweet = np.array(list(zip_longest(*val_tweet, fillvalue = np.zeros(300)))).T
val_tweet = val_tweet.transpose([1, 2, 0])
print(val_tweet.shape)

(689, 5, 300)


In [115]:
test_target = []
count = 0
for words in cleaned_target_token_test:
  vector_list = [model[word] for word in words if word in model.vocab]
  test_target.append(vector_list)

from itertools import zip_longest
test_target = np.array(list(zip_longest(*test_target, fillvalue = np.zeros(300)))).T
test_target = test_target.transpose([1, 2, 0])
print(test_target.shape)

(707, 2, 300)


In [116]:
val_target = []
count = 0
for words in cleaned_target_token_val:
  vector_list = [model[word] for word in words if word in model.vocab]
  val_target.append(vector_list)

from itertools import zip_longest
val_target = np.array(list(zip_longest(*val_target, fillvalue = np.zeros(300)))).T
val_target = val_target.transpose([1, 2, 0])
print(val_target.shape)

(689, 2, 300)


In [117]:
test_stance = pd.DataFrame.from_records(test_df['Stance'].to_numpy()).to_numpy()
train_stance = pd.DataFrame.from_records(balanced_train_df['Stance'].to_numpy()).to_numpy()
val_stance = pd.DataFrame.from_records(val_df['Stance'].to_numpy()).to_numpy()
print(train_stance.shape)

(1811, 3)


In [118]:
batch_size = 100

In [119]:
print(train_tweet.shape)
print(train_target.shape)
print(train_stance.shape)

(1811, 19, 300)
(1811, 4, 300)
(1811, 3)


In [120]:
train_ds = TensorDataset(torch.from_numpy(train_tweet), torch.from_numpy(train_target), torch.from_numpy(train_stance))
test_ds = TensorDataset(torch.from_numpy(test_tweet), torch.from_numpy(test_target), torch.from_numpy(test_stance))
val_ds = TensorDataset(torch.from_numpy(val_tweet), torch.from_numpy(val_target), torch.from_numpy(val_stance))

In [121]:
train_dl = DataLoader(train_ds, shuffle = True, batch_size = batch_size, drop_last = True)
test_dl = DataLoader(test_ds, shuffle = False, batch_size = batch_size, drop_last = True)
val_dl = DataLoader(val_ds, shuffle = False, batch_size = batch_size, drop_last = True)
print(len(train_dl.dataset))

1811


In [122]:
class LSTM_target(nn.Module):
  def __init__(self,  embedding_dim, hidden_dim):
    super().__init__()
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, bidirectional = True)
      
  def forward(self, x, hidden):
    out, hncn = self.lstm(x, hidden)
    hn, cn = hncn
    return cn

  def init_hidden(self):
    return (torch.zeros(2, batch_size, hidden_dim), torch.zeros(2, batch_size, hidden_dim))

In [123]:
class LSTM_twitter(nn.Module):
  def __init__(self,  embedding_dim, hidden_dim):
    super().__init__()
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first = True, bidirectional = True)
    self.W = nn.Linear(2 * hidden_dim, 3, bias = False)
      
  def forward(self, x, hidden):
    out, hncn = self.lstm(x, hidden)
    hn, cn = hncn
    hn = hn.reshape(batch_size, 2 * hidden_dim)  
    c = self.W(hn)  
    return torch.tanh(c)

  def init_hidden(self):
    return torch.zeros(2, batch_size, hidden_dim)

In [124]:
embedding_dim, hidden_dim = 300, 32
lstm_target = LSTM_target( embedding_dim, hidden_dim)
lstm_twitter = LSTM_twitter(embedding_dim, hidden_dim)

In [125]:
lstm_target.to(device)
lstm_twitter.to(device)

LSTM_twitter(
  (lstm): LSTM(300, 32, batch_first=True, bidirectional=True)
  (W): Linear(in_features=64, out_features=3, bias=False)
)

In [126]:
criterion = nn.CrossEntropyLoss()
optimizer1 = torch.optim.Adam(lstm_target.parameters(), lr = 0.1)
optimizer2 = torch.optim.Adam(lstm_twitter.parameters(), lr = 0.1)

In [127]:
from sklearn.metrics import f1_score
epochs = 5000
losses = []
for e in range(epochs):
  loss1 = 0
  num = 0
  for batch_idx, batch in enumerate(train_dl):        
    tweet, target, stance = batch
    target = target.float().to(device)
    tweet = tweet.float().to(device)
    stance = stance.float().to(device)
    with torch.set_grad_enabled(True):
      optimizer1.zero_grad()
      optimizer2.zero_grad()
      h0, c0 =  lstm_target.init_hidden()
      h0 = h0.to(device)
      c0 = c0.to(device)
      ct = lstm_target(target, (h0, c0))
      ht = lstm_twitter.init_hidden()
      ht = ht.to(device)
      c = lstm_twitter(tweet, (ht, ct)) 
      loss = criterion(c, stance.argmax(dim = 1))
      loss.backward()
      optimizer2.step()
      optimizer1.step()
      loss1 += loss.item()
      num += 1
  if e % 50 == 0:
    print('Epochs : {}, Loss : {}'.format(e, loss1 / num))
    losses.append(loss.item())
    total_length = 0
    total_right = 0
    for batch_idx, batch in enumerate(val_dl):
      tweet, target, stance = batch
      target = target.float().to(device)
      tweet = tweet.float().to(device)
      stance = stance.float().to(device)
      c = lstm_twitter(tweet, (h0, ct)) 
      total_length += 100
      total_right += f1_score(c.argmax(dim = 1).cpu().int().numpy(), stance.argmax(dim = 1).cpu().int().numpy(), average = 'weighted')
    print('The f1_score for the validation data is ', 100 * (total_right / total_length))

Epochs : 0, Loss : 1.1549274656507704
The f1_score for the validation data is  0.3653305609242195
Epochs : 50, Loss : 1.097502589225769
The f1_score for the validation data is  0.2501702974044331
Epochs : 100, Loss : 1.0981700089242723
The f1_score for the validation data is  0.2501702974044331
Epochs : 150, Loss : 1.0985576311747234
The f1_score for the validation data is  0.3653305609242195
Epochs : 200, Loss : 1.0981386303901672
The f1_score for the validation data is  0.3099776229175434
Epochs : 250, Loss : 1.0982853372891743
The f1_score for the validation data is  0.3782824839391259
Epochs : 300, Loss : 1.0987718635135226
The f1_score for the validation data is  0.4461197291107992
Epochs : 350, Loss : 1.0982937547895644
The f1_score for the validation data is  0.4461197291107992
Epochs : 400, Loss : 1.0986123945977952
The f1_score for the validation data is  0.6803388393398371
Epochs : 450, Loss : 1.0986122952567206
The f1_score for the validation data is  0.6803388393398371
Epoc

In [128]:
from sklearn.metrics import f1_score
batch_acc = []
total_length = 0
total_right = 0
new = []
for batch_idx, batch in enumerate(test_dl):
  h0, c0 = lstm_target.init_hidden()
  h0 = h0.to(device)
  c0 = c0.to(device)
  tweet, target, stance = batch
  target = target.float().to(device)
  tweet = tweet.float().to(device)
  stance = stance.float().to(device)
  cnt = 0
  with torch.set_grad_enabled(False):
    optimizer1.zero_grad()
    optimizer2.zero_grad()
    ct = lstm_target(target, (h0, c0))
    ht = lstm_twitter.init_hidden()
    ht = ht.to(device)
    c = lstm_twitter(tweet, (h0, ct)) 
    total_length += 100
    total_right += f1_score(c.argmax(dim = 1).cpu().int().numpy(), stance.argmax(dim = 1).cpu().int().numpy(), average = 'weighted')
    print(100 * f1_score(c.argmax(dim = 1).cpu().int().numpy(), stance.argmax(dim = 1).cpu().int().numpy(), average = 'weighted'))
print('The f1_score for the test data is ', 100 * (total_right / total_length))

21.428571428571423
34.710743801652896
93.61702127659575
77.30061349693253
87.00564971751412
19.819819819819816
40.0
The f1_score for the test data is  0.5341177422015522
